In [1]:
import numpy as np
from typing import List

import torch
import math
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
import argparse
from tqdm import tqdm

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from source import *

Running on: cpu


# Set-up

In [2]:
# Step RelU
# Version for the eta in the definition of a radial rescaling activation

def stepReLU_eta(r):
    if r.shape == torch.Size([]):
        if r < 1:
            return 1e-6
        return r
    else:
        for i in range(len(r)):
            if r[i] < 1:
                r[i] = 1e-6
        return r

In [3]:
# Create uniform random noise in the unit d-ball
def generate_noise(m, r, d=28*28):
    '''m is the number of samples, r is the radius
    d is the total dimension, which is 28*28 for MNIST'''
    
    u = np.random.multivariate_normal(np.zeros(d),np.eye(d),m)  # an array of d normally distributed random variables
    norm=np.sum(u**2, axis = 1) **(0.5)
    norm = norm.reshape(m,1)
    rands = np.random.uniform(size=m)**(1.0/d)
    rands = rands.reshape(m,1)
    return r*rands*u/norm

# Note: need to do the following before adding to a sample:
# torch.tensor(generate_noise(m,radius,d)).reshape(m,1,28,28)

In [4]:
# Calculate distances
def smallest_distance(x: torch.tensor) -> float:
    radius = float('inf')
    for i in range(len(x)):
        for j in range(i):
            if torch.linalg.norm(x[i] - x[j]).item() < radius:
                radius = torch.linalg.norm(x[i] - x[j]).item()
    return radius

# Get MNIST

In [5]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

In [6]:
batch_size = 128
train_dataloader = DataLoader(training_data, batch_size, shuffle=True)

In [7]:
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

if False:
    img = train_features[0].squeeze()
    label = train_labels[0]
    plt.imshow(img, cmap="gray")
    plt.show()
    print(f"Label: {label}")

Feature batch shape: torch.Size([128, 1, 28, 28])
Labels batch shape: torch.Size([128])


# Select threes and add noise

In [8]:
## Select the threes
def add_noise(label, n=5, m=100, verbose=False):
    '''label is one of 0,1,2,3,4,5,6,7,8,9;
    n is the number of original images; 
    m is the number of noisy samples per original image;
    may want to make the 2.5 definition of the radius into a hyperparameter
    '''
    
    # Get a selection of data with the same label, crop it to size n
    selection = train_features[train_labels == label]
    selection = selection[:n]
    assert selection.shape[0] == n, "Too few of this label; take another batch"
    if verbose:
        plt.imshow(selection[2].squeeze(), cmap="gray")
        plt.show()
    
    radius = smallest_distance(selection)/2.5
    noise = torch.tensor(generate_noise(m,radius,d=28*28)).reshape(m,1,28,28)
    
    noisy_samples = torch.Tensor(torch.Size([int(n*m), 1, 28, 28]))
    noisy_labels = torch.Tensor(torch.Size([n*m, n]))
    for i in range(n):
        for j in range(m):
            noisy_samples[i*n + j]= selection[i] + noise[j]   
            noisy_labels[i*m + j]=  torch.eye(n)[i]
    
    if verbose:
        plt.imshow(selection[0][0], cmap="gray")
        plt.show()
        plt.imshow(noisy_samples[0][0], cmap="gray")
        plt.show()
        plt.imshow(noisy_samples[1][0], cmap="gray")
    
    return noisy_samples, noisy_labels

# Alternative for the labels:
# torch.eye(n).repeat_interleave(m, dim=0)

In [9]:
print('Enter the number of original images:')
num_samples = input()

print('Enter the number of noisy copies of each:')
m_copies = input()

Enter the number of original images:
5
Enter the number of noisy copies of each:
5


In [10]:
noisy_threes, noisy_labels = add_noise(label=3, n=int(num_samples), m=int(m_copies), verbose =False)
print(noisy_threes.shape, noisy_labels.shape)

torch.Size([25, 1, 28, 28]) torch.Size([25, 5])


In [11]:
noisy_threes_flat = noisy_threes.flatten(1)

# Train with the noise

In [12]:
d=28*28
dim_vector = [d, d+1, d+2, d+3,1]

radnet = RadNet(eta=torch.sigmoid, dims=dim_vector, has_bias=False)

In [13]:
model_trained, model_losses = training_loop(
    n_epochs = 2000, 
    learning_rate = 0.05,
    model = radnet,
    params = list(radnet.parameters()),
    x_train = noisy_threes_flat,
    y_train = noisy_labels,
    verbose=True)

Epoch 1, Loss 0.197939
Epoch 500, Loss 0.160002
Epoch 1000, Loss 0.160001
Epoch 1500, Loss 0.160001
Epoch 2000, Loss 0.160000


# Train baseline

In [14]:
relu_net = torch.nn.Sequential(
    torch.nn.Linear(28*28, dim_vector[1]),
    torch.nn.ReLU(),
    torch.nn.Linear(dim_vector[1], dim_vector[2]),
    torch.nn.ReLU(),
    torch.nn.Linear(dim_vector[2], dim_vector[3]),
    torch.nn.ReLU(),
    torch.nn.Linear(dim_vector[3],1)
    )
print(relu_net)

Sequential(
  (0): Linear(in_features=784, out_features=785, bias=True)
  (1): ReLU()
  (2): Linear(in_features=785, out_features=786, bias=True)
  (3): ReLU()
  (4): Linear(in_features=786, out_features=787, bias=True)
  (5): ReLU()
  (6): Linear(in_features=787, out_features=1, bias=True)
)


In [15]:
relu_model_trained, relu_model_losses = training_loop(
    n_epochs = 2000, 
    learning_rate = 0.05,
    model = relu_net,
    params = list(relu_net.parameters()),
    x_train = noisy_threes_flat,
    y_train = noisy_labels,
    verbose=True)

Epoch 1, Loss 0.187763
Epoch 500, Loss 0.160002
Epoch 1000, Loss 0.160001
Epoch 1500, Loss 0.160000
Epoch 2000, Loss 0.160000


# Network for learning all of MNIST

In [16]:
train_features_flat = train_features.flatten(1)
train_features_flat.shape

torch.Size([128, 784])

In [17]:
radnet = RadNet(eta=torch.sigmoid, dims=[28*28,28*28, 28 , 28,1], has_bias=False)

In [18]:
model_trained, model_losses = training_loop(
    n_epochs = 3000, 
    learning_rate = 0.05,
    model = radnet,
    params = list(radnet.parameters()),
    x_train = train_features_flat,
    y_train = train_labels,
    verbose=True)

Epoch 1, Loss 27.101681
Epoch 500, Loss 8.545847
Epoch 1000, Loss 8.545847
Epoch 1500, Loss 8.545846
Epoch 2000, Loss 8.545847
Epoch 2500, Loss 8.545847
Epoch 3000, Loss 8.545846


# Scraps

In [19]:
# Calculate distances
radius = float('inf')
for i in range(n):
    for j in range(i+1,n):
        if torch.linalg.norm(threes[i] - threes[j]).item() < radius:
            radius = torch.linalg.norm(threes[i] - threes[j]).item()
radius = radius/2.5
radius

NameError: name 'n' is not defined

In [ ]:
noisy_threes = torch.Tensor(torch.Size([int(n*m), 1, 28, 28]))
noisy_labels = torch.Tensor(torch.Size([n*m, n]))
for i in range(n):
    for j in range(m):
        noisy_threes[i*n + j]= threes[i] + noise[j]   
        noisy_labels[i*m + j]=  torch.eye(n)[i]
        


if False:
    print(noisy_threes.shape, noisy_labels.shape)
if False:
    plt.imshow(threes[0][0], cmap="gray")
    plt.show()
    plt.imshow(noisy_threes[0][0], cmap="gray")
    plt.show()
    plt.imshow(noisy_threes[1][0], cmap="gray")